# Image Augmentation

1. Image resize

In [5]:
import cv2
import os

def resize_image_with_aspect_ratio_and_adjust_bboxes(image_path, bbox_path, output_folder, target_size=(100, 100)):
    # 이미지 읽기
    image = cv2.imread(image_path)
    h, w, _ = image.shape

    # 비율 유지하며 이미지 리사이즈
    scale = min(target_size[0] / h, target_size[1] / w)
    new_w = int(w * scale)
    new_h = int(h * scale)

    resized_image = cv2.resize(image, (new_w, new_h))

    # 패딩 추가하여 target_size에 맞추기
    delta_w = target_size[1] - new_w
    delta_h = target_size[0] - new_h
    top, bottom = delta_h // 2, delta_h - (delta_h // 2)
    left, right = delta_w // 2, delta_w - (delta_w // 2)

    color = [0, 0, 0]  # 검은색 패딩
    padded_image = cv2.copyMakeBorder(resized_image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    # 바운딩 박스 읽기 (YOLO 형식)
    bboxes = []
    with open(bbox_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            label = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])

            # 기존 바운딩 박스 좌표를 리사이즈된 이미지에 맞게 조정
            new_x_center = (x_center * w * scale + left) / target_size[1]
            new_y_center = (y_center * h * scale + top) / target_size[0]
            new_width = width * scale
            new_height = height * scale

            bboxes.append([label, new_x_center, new_y_center, new_width, new_height])

    # 리사이즈된 이미지 저장 경로
    resized_image_path = os.path.join(output_folder, os.path.splitext(os.path.basename(image_path))[0] + '_resized.jpg')
    cv2.imwrite(resized_image_path, padded_image)

    # 조정된 바운딩 박스 저장 경로
    resized_bbox_path = os.path.join(output_folder, os.path.splitext(os.path.basename(bbox_path))[0] + '_resized.txt')
    with open(resized_bbox_path, 'w') as file:
        for bbox in bboxes:
            file.write(f"{bbox[0]} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f} {bbox[4]:.6f}\n")

    return resized_image_path, resized_bbox_path

# 예시 경로
data_path = 'cropped_images2/'  # 데이터가 있는 디렉토리 경로
output_folder = 'resized_images_60x60'  # 이미지를 저장할 경로

# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 데이터 디렉토리 내 모든 파일에 대해 처리
image_extensions = ('.jpg', '.jpeg', '.png')  # 지원하는 이미지 파일 확장자
for filename in os.listdir(data_path):
    if filename.endswith(image_extensions):
        image_path = os.path.join(data_path, filename)
        bbox_path = os.path.splitext(image_path)[0] + '.txt'  # 동일한 이름의 바운딩 박스 파일

        if os.path.exists(bbox_path):
            resize_image_with_aspect_ratio_and_adjust_bboxes(image_path, bbox_path, output_folder, target_size=(100, 100))
            print(f"Resized image and bounding box saved for {image_path}")
        else:
            print(f"Bounding box file not found for {image_path}")


Resized image and bounding box saved for cropped_images2/ApurtranTablet150mg(Irbesartan)_crop_0_0.jpg
Resized image and bounding box saved for cropped_images2/ApurtranTablet150mg(Irbesartan)_crop_0_1.jpg
Resized image and bounding box saved for cropped_images2/BalsanTablet80mg(Valsartan)_crop_1_0.jpg
Resized image and bounding box saved for cropped_images2/BalsanTablet80mg(Valsartan)_crop_1_1.jpg
Resized image and bounding box saved for cropped_images2/BalthrepTablet160mg(Valsartan)_crop_2_0.jpg
Resized image and bounding box saved for cropped_images2/BalthrepTablet160mg(Valsartan)_crop_2_1.jpg
Resized image and bounding box saved for cropped_images2/DiosartanTablet160mg(Valsartan)_crop_3_0.jpg
Resized image and bounding box saved for cropped_images2/DiosartanTablet160mg(Valsartan)_crop_3_1.jpg
Resized image and bounding box saved for cropped_images2/EscitalTablet5mg(EscitalopramOxalate)_crop_4_0.jpg
Resized image and bounding box saved for cropped_images2/EscitalTablet5mg(Escitalopram